In [2]:
%pip install shapely

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
    --------------------------------------- 0.0/1.4 MB 146.3 kB/s eta 0:00:10
    --------------------------------------- 0.0/1.4 MB 146.3 kB/s eta 0:00:10
    --------------------------------------- 0.0/1.4 MB 146.3 kB/s eta 0:00:10
    --------------------------------------- 0.0/1.4 MB 146.3 kB/s eta 0:00:10
    --------------------------------------- 0.0/1.4 MB 146.3 kB/s eta 0:00:10
    --------------------------------------- 0.0/1.4 MB 146.3 kB/s eta 0:00:10
    --------------------------------------- 0.0/1.4 MB 146.3 kB/s eta 0:00:10
    --------------------------------------- 0.0/1.4 MB 146.3 kB/s eta 0:00:10
    ---------------------------

In [3]:
%pip install pyproj

   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/6.1 MB 145.2 kB/s eta 0:00:42
   ---------------------------------------- 0.0/6.1 MB 145.2 kB/s eta 0:00:42
   ---------------------------------------- 0.0/6.1 MB 145.2 kB/s eta 0:00:42
   ---------------------------------------- 0.0/6.1 MB 145.2 kB/s eta 0:00:42
   ---------------------------------------- 0.0/6.1 MB 145.2 kB/s eta 0:00:42
   ---------------------------------------- 0.0/6.1 MB 145.2 kB/s eta 0:00:42
   ---------------------------------------- 0.0/6.1 MB 145.2 kB/s eta 0:00:42
   -------------------------------------

In [8]:
import json
from shapely.geometry import MultiPolygon, Polygon
from pyproj import Proj, transform

# Define the UTM Zone 48N projection (EPSG:32648)
proj_utm = Proj(proj='utm', zone=48, ellps='WGS84', south=False)
proj_latlon = Proj(proj='latlong', datum='WGS84')

def calculate_centroid(coordinates):
    # Convert coordinates to a MultiPolygon object
    polygons = [Polygon(polygon[0]) for polygon in coordinates]
    multipolygon = MultiPolygon(polygons)
    
    # Calculate the centroid of the MultiPolygon in UTM
    centroid_utm = multipolygon.centroid
    
    # Convert centroid from UTM to Latitude/Longitude
    longitude, latitude = transform(proj_utm, proj_latlon, centroid_utm.x, centroid_utm.y)
    
    return {"longitude": longitude, "latitude": latitude}

def extract_district_centroids(input_file, output_file):
    with open(input_file, 'r') as f:
        data = json.load(f)
    
    centroids = []
    for feature in data['features']:
        district_name = feature['properties']['DIS_NAME']
        coordinates = feature['geometry']['coordinates']
        centroid = calculate_centroid(coordinates)
        centroids.append({
            "district_name": district_name,
            "centroid": centroid
        })
    
    with open(output_file, 'w') as f:
        json.dump(centroids, f, indent=4)

# Input and output file paths
input_file = 'district.geojson' 
output_file = 'district_centroids.geojson'  

# Calculate centroids and save to a new JSON file
extract_district_centroids(input_file, output_file)

C:\Users\SO KIMLANG\AppData\Local\Temp\ipykernel_20808\3405399721.py:18: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude, latitude = transform(proj_utm, proj_latlon, centroid_utm.x, centroid_utm.y)


In [2]:
%pip install geopy

Note: you may need to restart the kernel to use updated packages.Collecting geopy
  Using cached geopy-2.4.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached geographiclib-2.0-py3-none-any.whl.metadata (1.4 kB)
Using cached geopy-2.4.1-py3-none-any.whl (125 kB)
Using cached geographiclib-2.0-py3-none-any.whl (40 kB)




[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: C:\Users\SO KIMLANG\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [5]:
import json
from shapely.geometry import MultiPolygon, Polygon
from pyproj import Proj, transform
from geopy.distance import geodesic

# Define the UTM Zone 48N projection (EPSG:32648)
proj_utm = Proj(proj='utm', zone=48, ellps='WGS84', south=False)
proj_latlon = Proj(proj='latlong', datum='WGS84')

# Define the distance threshold for considering neighbors
NEIGHBOR_THRESHOLD_KM = 30

# Coordinates for Tuol Kouk
tuol_kouk_coords = {"longitude": 104.899, "latitude": 11.571}

def calculate_centroid(coordinates):
    # Convert coordinates to a MultiPolygon object
    polygons = [Polygon(polygon[0]) for polygon in coordinates]
    multipolygon = MultiPolygon(polygons)
    
    # Calculate the centroid of the MultiPolygon in UTM
    centroid_utm = multipolygon.centroid
    
    # Convert centroid from UTM to Latitude/Longitude
    longitude, latitude = transform(proj_utm, proj_latlon, centroid_utm.x, centroid_utm.y)
    
    return {"longitude": longitude, "latitude": latitude}

def calculate_distance(coord1, coord2):
    return geodesic((coord1["latitude"], coord1["longitude"]), (coord2["latitude"], coord2["longitude"])).km

def extract_district_centroids(input_file, output_file):
    with open(input_file, 'r') as f:
        data = json.load(f)
    
    districts_data = []
    for feature in data['features']:
        district_name = feature['properties']['DIS_NAME']
        dis_code = feature['properties']['DIS_CODE']
        coordinates = feature['geometry']['coordinates']
        
        centroid = calculate_centroid(coordinates)
        distance_from_tuol_kouk = calculate_distance(centroid, tuol_kouk_coords)
        
        districts_data.append({
            "district_name": district_name,
            "centroid": centroid,
            "DIS_CODE": dis_code,
            "distance_from_tuol_kouk": distance_from_tuol_kouk,
            "neighbors": []  # To be filled later
        })
    
    # Determine neighbors based on distance
    for i, district in enumerate(districts_data):
        for j, other_district in enumerate(districts_data):
            if i != j:
                distance = calculate_distance(district["centroid"], other_district["centroid"])
                if distance <= NEIGHBOR_THRESHOLD_KM:
                    district["neighbors"].append(other_district["district_name"])
    
    # Save the updated district data to a new JSON file
    with open(output_file, 'w') as f:
        json.dump(districts_data, f, indent=4)

# Input and output file paths
input_file = 'district.geojson' 
output_file = 'district_centroids.json'  

# Calculate centroids, distances, and neighbors, then save to a new JSON file
extract_district_centroids(input_file, output_file)

print("New JSON file created with centroid, DIS_CODE, neighbors, and distances from Tuol Kouk.")

C:\Users\SO KIMLANG\AppData\Local\Temp\ipykernel_5464\344857049.py:25: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  longitude, latitude = transform(proj_utm, proj_latlon, centroid_utm.x, centroid_utm.y)


New JSON file created with centroid, DIS_CODE, neighbors, and distances from Tuol Kouk.


In [3]:
import json
from shapely.geometry import shape, MultiPolygon, Polygon
from pyproj import Proj, Transformer
from geopy.distance import geodesic
import requests

# Define the UTM Zone 48N projection (EPSG:32648)
proj_utm = Proj(proj='utm', zone=48, ellps='WGS84', south=False)
proj_latlon = Proj(proj='latlong', datum='WGS84')
transformer = Transformer.from_proj(proj_utm, proj_latlon, always_xy=True)

# Google Distance Matrix API setup
API_KEY = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
BASE_URL = 'https://maps.googleapis.com/maps/api/distancematrix/json'

# Coordinates for Tuol Kouk
tuol_kouk_coords = {"longitude": 104.899, "latitude": 11.571}

def calculate_centroid(coordinates):
    try:
        # Convert coordinates to a MultiPolygon object
        polygons = [Polygon(polygon[0]) for polygon in coordinates]
        multipolygon = MultiPolygon(polygons)
        
        # Calculate the centroid of the MultiPolygon in UTM
        centroid_utm = multipolygon.centroid
        
        # Convert centroid from UTM to Latitude/Longitude
        longitude, latitude = transformer.transform(centroid_utm.x, centroid_utm.y)
        
        return {"longitude": longitude, "latitude": latitude}
    except Exception as e:
        print(f"Error calculating centroid: {e}")
        return {"longitude": None, "latitude": None}

def calculate_distance(coord1, coord2):
    return geodesic((coord1["latitude"], coord1["longitude"]), (coord2["latitude"], coord2["longitude"])).km

def get_actual_distance(origin, destination):
    try:
        params = {
            'origins': f'{origin["latitude"]},{origin["longitude"]}',
            'destinations': f'{destination["latitude"]},{destination["longitude"]}',
            'key': API_KEY,
            'mode': 'driving'
        }
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()  # Raise an error for bad responses
        
        result = response.json()

        # Print the full response to understand the structure
        print("API Response:", json.dumps(result, indent=2))

        # Check if the response structure contains the necessary elements
        if "rows" in result and len(result["rows"]) > 0:
            elements = result["rows"][0]["elements"]
            if len(elements) > 0 and "status" in elements[0] and elements[0]["status"] == "OK":
                distance_meters = elements[0]["distance"]["value"]
                distance_km = distance_meters / 1000.0  # Convert meters to kilometers
                return distance_km
            else:
                print(f"API response error: {elements[0].get('status', 'Unknown Error')}")
                return None  # Use None to indicate failure
        else:
            print("Unexpected API response structure.")
            return None
    except Exception as e:
        print(f"Error fetching distance: {e}")
        return None  # Use None to indicate failure

def extract_district_data(input_file, output_file):
    with open(input_file, 'r') as f:
        data = json.load(f)
    
    districts_data = []
    for feature in data['features']:
        district_name = feature['properties']['DIS_NAME']
        dis_code = feature['properties']['DIS_CODE']
        geometry = shape(feature['geometry'])  # Use shapely to create a shape object

        centroid = calculate_centroid(feature['geometry']['coordinates'])
        if centroid["longitude"] is None or centroid["latitude"] is None:
            continue  # Skip this district if centroid calculation failed
        
        distance_from_tuol_kouk = calculate_distance(centroid, tuol_kouk_coords)
        actual_distance_from_tuol_kouk = get_actual_distance(tuol_kouk_coords, centroid)
        
        districts_data.append({
            "district_name": district_name,
            "centroid": centroid,
            "DIS_CODE": dis_code,
            "geometry": geometry,  # Store the geometry
            "distance_from_tuol_kouk": distance_from_tuol_kouk,
            "actual_distance_from_tuol_kouk": actual_distance_from_tuol_kouk,
            "neighbors": []  # To be filled later
        })
    
    # Determine neighbors using spatial adjacency
    for i, district in enumerate(districts_data):
        for j, other_district in enumerate(districts_data):
            if i != j and district['geometry'].touches(other_district['geometry']):
                if other_district["district_name"] not in district["neighbors"]:
                    district["neighbors"].append(other_district["district_name"])
                if district["district_name"] not in other_district["neighbors"]:
                    other_district["neighbors"].append(district["district_name"])
    
    # Save the updated district data to a new JSON file
    for district in districts_data:
        district.pop("geometry")  # Remove geometry before saving to JSON
        
    with open(output_file, 'w') as f:
        json.dump(districts_data, f, indent=4)

# Input and output file paths
input_file = 'district.geojson' 
output_file = 'district_centroids_with_distances.json'

# Calculate centroids, distances, and neighbors, then save to a new JSON file
extract_district_data(input_file, output_file)

print("New JSON file created with centroid, DIS_CODE, neighbors, actual travel distances, and distances from Tuol Kouk.")

API Response: {
  "destination_addresses": [
    "Unnamed Road, Cambodia"
  ],
  "origin_addresses": [
    "HVCX+CGG, Mohasenapramuk Kim Il Sung Blvd (289), Phnom Penh, Cambodia"
  ],
  "rows": [
    {
      "elements": [
        {
          "distance": {
            "text": "345 km",
            "value": 345158
          },
          "duration": {
            "text": "5 hours 12 mins",
            "value": 18706
          },
          "status": "OK"
        }
      ]
    }
  ],
  "status": "OK"
}
API Response: {
  "destination_addresses": [
    "Unnamed Road, Cambodia"
  ],
  "origin_addresses": [
    "HVCX+CGG, Mohasenapramuk Kim Il Sung Blvd (289), Phnom Penh, Cambodia"
  ],
  "rows": [
    {
      "elements": [
        {
          "distance": {
            "text": "407 km",
            "value": 406936
          },
          "duration": {
            "text": "6 hours 34 mins",
            "value": 23617
          },
          "status": "OK"
        }
      ]
    }
  ],
  "status": "

In [5]:
%pip install geopy

  Using cached geopy-2.4.1-py3-none-any.whl.metadata (6.8 kB)
  Using cached geographiclib-2.0-py3-none-any.whl.metadata (1.4 kB)
Using cached geopy-2.4.1-py3-none-any.whl (125 kB)
Using cached geographiclib-2.0-py3-none-any.whl (40 kB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import json
from shapely.geometry import shape, MultiPolygon, Polygon
from pyproj import Proj, Transformer
from geopy.distance import geodesic
import requests

# Define the UTM Zone 48N projection (EPSG:32648)
proj_utm = Proj(proj='utm', zone=48, ellps='WGS84', south=False)
proj_latlon = Proj(proj='latlong', datum='WGS84')
transformer = Transformer.from_proj(proj_utm, proj_latlon, always_xy=True)

# Google Distance Matrix API setup
API_KEY = 'XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX'
BASE_URL = 'https://maps.googleapis.com/maps/api/distancematrix/json'

# Coordinates for Tuol Kouk
tuol_kouk_coords = {"longitude": 104.899, "latitude": 11.571}

def calculate_centroid(coordinates):
    try:
        # Convert coordinates to a MultiPolygon object
        polygons = [Polygon(polygon[0]) for polygon in coordinates]
        multipolygon = MultiPolygon(polygons)
        
        # Calculate the centroid of the MultiPolygon in UTM
        centroid_utm = multipolygon.centroid
        
        # Convert centroid from UTM to Latitude/Longitude
        longitude, latitude = transformer.transform(centroid_utm.x, centroid_utm.y)
        
        return {"longitude": longitude, "latitude": latitude}
    except Exception as e:
        print(f"Error calculating centroid: {e}")
        return {"longitude": None, "latitude": None}

def calculate_distance(coord1, coord2):
    return geodesic((coord1["latitude"], coord1["longitude"]), (coord2["latitude"], coord2["longitude"])).km

def get_actual_distance(origin, destination):
    try:
        params = {
            'origins': f'{origin["latitude"]},{origin["longitude"]}',
            'destinations': f'{destination["latitude"]},{destination["longitude"]}',
            'key': API_KEY,
            'mode': 'driving'
        }
        response = requests.get(BASE_URL, params=params)
        response.raise_for_status()  # Raise an error for bad responses
        
        result = response.json()

        # Check if the response structure contains the necessary elements
        if "rows" in result and len(result["rows"]) > 0:
            elements = result["rows"][0]["elements"]
            if len(elements) > 0 and "status" in elements[0] and elements[0]["status"] == "OK":
                distance_meters = elements[0]["distance"]["value"]
                distance_km = distance_meters / 1000.0  # Convert meters to kilometers
                return distance_km
            else:
                print(f"API response error: {elements[0].get('status', 'Unknown Error')}")
                return None  # Use None to indicate failure
        else:
            print("Unexpected API response structure.")
            return None
    except Exception as e:
        print(f"Error fetching distance: {e}")
        return None  # Use None to indicate failure

def extract_district_data(input_file, output_file):
    with open(input_file, 'r') as f:
        data = json.load(f)
    
    districts_data = []
    for feature in data['features']:
        district_name = feature['properties']['DIS_NAME']
        dis_code = feature['properties']['DIS_CODE']
        geometry = shape(feature['geometry'])  # Use shapely to create a shape object

        centroid = calculate_centroid(feature['geometry']['coordinates'])
        if centroid["longitude"] is None or centroid["latitude"] is None:
            continue  # Skip this district if centroid calculation failed
        
        distance_from_tuol_kouk = calculate_distance(centroid, tuol_kouk_coords)
        actual_distance_from_tuol_kouk = get_actual_distance(tuol_kouk_coords, centroid)
        
        districts_data.append({
            "district_name": district_name,
            "centroid": centroid,
            "DIS_CODE": dis_code,
            "geometry": geometry,  # Store the geometry
            "distance_from_tuol_kouk": distance_from_tuol_kouk,
            "actual_distance_from_tuol_kouk": actual_distance_from_tuol_kouk,
            "neighbors": [],  # To be filled later
            "distances_to_neighbors": {}  # To store distances to each neighbor
        })
    
    # Determine neighbors using spatial adjacency
    for i, district in enumerate(districts_data):
        for j, other_district in enumerate(districts_data):
            if i != j and district['geometry'].touches(other_district['geometry']):
                if other_district["district_name"] not in district["neighbors"]:
                    district["neighbors"].append(other_district["district_name"])
                if district["district_name"] not in other_district["neighbors"]:
                    other_district["neighbors"].append(district["district_name"])

                # Calculate the distance from the current district to its neighbor using the API
                distance_to_neighbor = get_actual_distance(district['centroid'], other_district['centroid'])
                
                if distance_to_neighbor is not None:
                    district["distances_to_neighbors"][other_district["district_name"]] = distance_to_neighbor
                    other_district["distances_to_neighbors"][district["district_name"]] = distance_to_neighbor
    
    # Save the updated district data to a new JSON file
    for district in districts_data:
        district.pop("geometry")  # Remove geometry before saving to JSON
        
    with open(output_file, 'w') as f:
        json.dump(districts_data, f, indent=4)

# Input and output file paths
input_file = 'district.geojson' 
output_file = 'district_centroids_with_distances.json'

# Calculate centroids, distances, and neighbors, then save to a new JSON file
extract_district_data(input_file, output_file)

print("New JSON file created with centroid, DIS_CODE, neighbors, actual travel distances, and distances from Tuol Kouk.")